In [561]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score

pd.set_option('display.max_columns', None)



# EX2. Comparison of different Credit Score Models performance on a public dataset

We compare different credit score models on teh database train.csv obtained by Kaggle at the link https://www.kaggle.com/competitions/credit-default-prediction-ai-big-data/data?select=test.csv. The name of the dataset is 'db_exercise_2_CSMs.csv'

0. split the dataset in training set and numerical set, with a proportion 0.75 - 0.25

1. EDA (exploratory data analysis):
    For the training set and the test set provides separately the following steps: 
    1. converting categorical data in numeric when possible; 
    2. missing values: substitute the missing value of every numerical variable with the median value on the dataset; for the categoric variable substitute the missing values with the most present value on the dataset. 
    3. outliers: for every numerical variable that takes continuous values: substitute the value beyond the 97th percentile and the 3rd percentile with the 97th percentile and the 3rd percentile. We say that a feature takes continuous values if it takes more than 1000 different values; 
    4. Data visualization:
        1. For every categorical feature provide the distribution of default: plot the distribution of the default on every value of every categoric variable
        2. Provide the correlation matrix between the numerical features. 

2. Apply Smote algorithm to the training set to manage the imbalanced classes.

3. Model estimation. First of all, exclude the feature "Credit Score" from the selection (in point 4. we are going to see the it has high correlation with the credit_default variable). For every model in the following list, provide the best hyper-parameters in terms of f1 score. To do so, apply GridSearchCV to the list of parameters gives in the following dictionary. Then fit the obtained model to the training set, both in the case in which the training set is not handled by Smote algorithm, as well as in the case in which it is:
    1. KNN;
    2. Logistic Regression;
    3. Decision Tree;
    4. Random Forest;
    5. Adaboost;
    6. XGBoost.
    

4. Model evaluation. Provide the following performance score. Compare the Roc curve of every model in the same figure. 
    1. Accuracy;
    2. Precision;
    3. Recall;
    4. F-score;
    5. ROC curve;
    6. AUC.





In [563]:
#Data: 

random_state = 100


In [562]:

dictio_parameters = {
                        'knn' : {'n_neighbors' : list(range(1,101,10))},
                        'logistic_regression': dict(zip(['tol', 'C', 'max_iter','solver'], 
                                                    [ 
                                                        [10**(-i) for i in range(1,7)],
                                                        [0.001,0.01,0.1,1,10,100],
                                                        [100*i for i in range(1,11)],
                                                        ['liblinear']
                                                        ])),
                        'decision_tree_classifier' : dict(zip(  ['max_depth','min_samples_split','min_samples_leaf'],
                                                    [   None,
                                                        [2,3,4,5],
                                                        [1,2,3,4]])),
                        'Random_forest_classifier' : dict(zip(['n_estimators','max_depth','min_samples_split','min_samples_leaf'],
                                                    [   [100,200,300],
                                                        None,
                                                        [2,3,4],
                                                        [1,2,3]] )),
                        'Adaboost_Classifier' : dict(zip(['n_estimators', 'learning_rate'], 
                                                    [   [100,200,300],
                                                        [0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3]])),
                        'XGboost_Classifier' : dict(zip(['n_estimators','max_depth','learning_rate','gamma'], 
                                                    [   [100,200,300],
                                                        None,
                                                        [0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3],
                                                        [1,0.1,0.01,0]
                                                        ]))}



In [564]:
#df = pd.read_csv('train.csv')
#df.to_csv('db_exercise_2_CSMs.csv')
df = pd.read_csv('db_exercise_2_CSMs.csv').drop(columns = ['Unnamed: 0'])

default_variable = 'Credit Default'
id_variable = 'Id'
